## Deep Research

One of the classic cross-business Agentic use cases! This is huge.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities. You can make use of this yourself!
            </span>
        </td>
    </tr>
</table>

In [1]:
# Imports

from agents import Agent, WebSearchTool, trace, Runner, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel
from dotenv import load_dotenv
import asyncio
import os
from IPython.display import display, Markdown
from pprint import pprint
import requests
load_dotenv(override=True)

# Constants

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

/Users/shreyas/gitrepos/ai_research/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern.

Costs are here: https://platform.openai.com/docs/pricing#web-search

## We will be making 4 Agents:

1. Search Agent - searches online given a search term using an OpenAI hosted tool
2. Planner Agent - given a query from the user, come up with searches
3. Report Agent - make a report on results
4. Push Agent - send a notification to the user's phone with a summary

## Our First Agent: Search Agent

Given a Search term, search for it on the internet and summarize results.

In [2]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4.1-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [3]:
message = "Find the best open source Trading models for forecasting BTC prices"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

Several open-source models have been developed for forecasting Bitcoin (BTC) prices, leveraging various machine learning and deep learning techniques. Notable among these are:

- **MRC-LSTM**: Combines Multi-scale Residual Convolutional Neural Networks (CNN) with Long Short-Term Memory (LSTM) networks to predict Bitcoin's closing prices. This hybrid approach effectively captures features across different time scales and learns long-term dependencies in financial time series data. ([arxiv.org](https://arxiv.org/abs/2105.00707?utm_source=openai))

- **CryptoMamba**: Utilizes a Mamba-based State Space Model (SSM) architecture designed to capture long-range dependencies in financial time series data. It has demonstrated enhanced generalizability across different market conditions, surpassing the limitations of previous models. ([arxiv.org](https://arxiv.org/abs/2501.01010?utm_source=openai))

- **TimeGPT**: A foundational model for time series forecasting and anomaly detection, TimeGPT is designed to handle financial time series data, including Bitcoin price predictions. It offers robust forecasting capabilities, uncertainty quantification, and anomaly detection, making it valuable for financial analysis and risk management. ([nixtla.dev](https://nixtla.dev/docs/use-cases-bitcoin_price_prediction?utm_source=openai))

- **PreBit**: A multimodal model that incorporates Twitter FinBERT embeddings to predict extreme price movements in Bitcoin. By combining social media sentiment analysis with technical indicators and correlated asset prices, PreBit aims to forecast significant market fluctuations. ([arxiv.org](https://arxiv.org/abs/2206.00648?utm_source=openai))

- **Helformer**: An attention-based deep learning model for cryptocurrency price forecasting, Helformer employs attention mechanisms to capture complex patterns in Bitcoin price data, enhancing prediction accuracy. ([journalofbigdata.springeropen.com](https://journalofbigdata.springeropen.com/articles/10.1186/s40537-025-01135-4?utm_source=openai))

These models are accessible through their respective repositories and publications, providing valuable resources for researchers and practitioners interested in Bitcoin price forecasting. 

Take a look at the trace

https://platform.openai.com/traces

## Our Second Agent: Planner Agent

Given a query, come up with 5 ideas for web searches that could be run.

Use Structured Outputs as our way to ensure the Agent provides what we need.

In [4]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 5

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# We use Pydantic objects to describe the Schema of the output

class WebSearchItem(BaseModel):
    reason: str
    "Your reasoning for why this search is important to the query."

    query: str
    "The search term to use for the web search."


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    """A list of web searches to perform to best answer the query."""

# We pass in the Pydantic object to ensure the output follows the schema

planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4.1-mini",
    output_type=WebSearchPlan,
)

In [5]:

message = "What are the most popular and successful open source frameworks for predicting BTC prices"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    pprint(result.final_output)

WebSearchPlan(searches=[WebSearchItem(reason='Identify leading open source frameworks specifically designed for Bitcoin price prediction.', query='open source frameworks for Bitcoin price prediction'), WebSearchItem(reason='Discover popular machine learning libraries frequently used for cryptocurrency price forecasting.', query='machine learning libraries for cryptocurrency price prediction'), WebSearchItem(reason='Find successful case studies or projects using open source tools to predict BTC prices.', query='successful open source projects predicting Bitcoin prices'), WebSearchItem(reason='Explore community discussions and recommendations on tools for BTC price prediction.', query='best open source tools for Bitcoin price forecasting'), WebSearchItem(reason='Identify frameworks with proven accuracy or popularity in BTC price prediction competitions or challenges.', query='top performing open source frameworks Bitcoin price prediction')])


## Our Third Agent: Writer Agent

Take the results of internet searches and make a report

In [6]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str
    """A short 2-3 sentence summary of the findings."""

    markdown_report: str
    """The final report"""

    follow_up_questions: list[str]
    """Suggested topics to research further"""


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

## Our Fourth Agent: push notification

Just to show how easy it is to make a tool!

I'm using a nifty product called PushOver - to set this up yourself, visit https://pushover.net

In [7]:
@function_tool
def push(message: str):
    """Send a push notification with this brief message"""
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)
    return {"status": "success"}

In [8]:
push

FunctionTool(name='push', description='Send a push notification with this brief message', params_json_schema={'properties': {'message': {'title': 'Message', 'type': 'string'}}, 'required': ['message'], 'title': 'push_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x109f2a160>, strict_json_schema=True)

In [9]:
INSTRUCTIONS = """You are a member of a research team and will be provided with a short summary of a report.
When you receive the report summary, you send a push notification to the user using your tool, informing them that research is complete,
and including the report summary you receive"""


push_agent = Agent(
    name="Push agent",
    instructions=INSTRUCTIONS,
    tools=[push],
    model="gpt-4.1-mini",
    model_settings=ModelSettings(tool_choice="required")
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [10]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### The next 2 functions write a report and send a push notification

In [11]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_push(report: ReportData):
    """ Use the push agent to send a notification to the user """
    print("Pushing...")
    result = await Runner.run(push_agent, report.short_summary)
    print("Push sent")
    return report

### Showtime!

In [12]:
query ="What are the most popular and successful open source BTC prediction frameworks"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_push(report)  
    print("Hooray!")
display(Markdown(report.markdown_report))

Starting research...
Planning searches...
Will perform 5 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Pushing...
Push sent
Hooray!


# Open Source Bitcoin Price Prediction Frameworks: A Comprehensive Overview

## Table of Contents  
1. [Introduction](#introduction)  
2. [Frameworks Overview](#frameworks-overview)  
   2.1 [Time Series Forecasting Models](#time-series-forecasting-models)  
   2.2 [Machine Learning Techniques](#machine-learning-techniques)  
   2.3 [Deep Learning Paradigms](#deep-learning-paradigms)  
   2.4 [Sentiment Analysis Models](#sentiment-analysis-models)  
3. [Comparative Analysis of Frameworks](#comparative-analysis-of-frameworks)  
4. [Case Studies](#case-studies)  
5. [Challenges and Limitations](#challenges-and-limitations)  
6. [Conclusion](#conclusion)  
7. [Future Directions](#future-directions)  
8. [References](#references)  

---  

## 1. Introduction  
In recent years, Bitcoin (BTC) has gained immense popularity and continues to attract the attention of investors and researchers alike. Price prediction is crucial for traders wishing to optimize their strategies in this volatile market. This report aims to explore the most effective open-source frameworks that have emerged for BTC price prediction, providing an in-depth view of their capabilities, methodologies, and applicability in real-world scenarios.

## 2. Frameworks Overview  
Several prominent families of frameworks have emerged for Bitcoin price forecasting, including time series models, statistical methods, and advanced machine-learning approaches. Below we categorize these frameworks to explore their unique features and methodologies.

### 2.1 Time Series Forecasting Models  
- **TimeGPT**: As a foundational model for time series forecasting, TimeGPT specializes in anomaly detection and is well-suited for Bitcoin price prediction. Its strength lies in its capability to process historical data patterns effectively ([Nixtla](https://nixtla.dev/docs/use-cases-bitcoin_price_prediction?utm_source=openai)).  

- **SARIMA and Facebook Prophet**: The Seasonal Autoregressive Integrated Moving Average (SARIMA) model is often combined with Prophet for effective trend extraction. This hybrid model benefits from both traditional statistical methodologies and modern machine learning techniques ([ScienceDirect](https://www.sciencedirect.com/science/article/abs/pii/S0040162523006236?utm_source=openai)).  

### 2.2 Machine Learning Techniques  
- **Ensemble Models**: Frameworks like the one developed by sakshamverma21 implement ensemble methods, combining Gradient Boosting, XGBoost, LightGBM, and LSTM for superior predictive performance ([GitHub - sakshamverma21](https://github.com/sakshamverma21/Bitcoin-Price-Prediction-Model?utm_source=openai)).  

- **Bayesian Regression**: Implementations such as the 'Bayesian regression for latent source model' allow for predicting Bitcoin price variations, enriching the analysis with statistical rigor ([GitHub - stavros0](https://github.com/stavros0/bitcoin-price-prediction?utm_source=openai)).  

### 2.3 Deep Learning Paradigms  
- **LSTM Neural Networks**: Long Short-Term Memory networks are widely utilized due to their efficacy in processing sequential data, particularly in predicting Bitcoin prices from historical data ([GitHub - Avinashrkrk](https://github.com/Avinashrkrk/Bitcoin-Price-Prediction?utm_source=openai)).  

- **MRC-LSTM**: This innovative approach combines Multi-scale Residual CNN with LSTM, achieving better performance in capturing long-term dependencies in the price data through systematic hyperparameter tuning ([arXiv](https://arxiv.org/abs/2105.00707?utm_source=openai)).  

### 2.4 Sentiment Analysis Models  
- **BTCPredictionWithTweets**: Utilizing sentiment analysis based on Twitter data, this framework integrates market sentiment to enhance Bitcoin price predictions, reflecting the influence of social media on market movements ([GitHub-Wiki](https://github-wiki-see.page/m/hupseb/BTCPredictionWithTweets/wiki/Bitcoin-Price-Prediction-using-Sentiment-Analysis-of-Twitter-Data?utm_source=openai)).  

- **LunarCrush**: By combining insights from social media platforms, this framework analyzes sentiment trends alongside market data to provide holistic price forecasts ([Blockchain Magazine](https://blockchainmagazine.com/best-reviews-of-quantify-crypto-terminal/?utm_source=openai)).  

## 3. Comparative Analysis of Frameworks  
A comparative analysis of the aforementioned frameworks reveals significant distinctions in efficacy and application:
- **Predictive Accuracy**: Hybrid models such as MRC-LSTM often outperform traditional methods like SARIMA due to their ability to learn complex and non-linear data relationships.
- **Implementation Complexity**: Models like LSTM require more computational resources and hyperparameter tuning compared to simpler approaches like Bayesian regression.
- **User Flexibility**: Tools like TimeGPT and CryptoAnalytics Toolkit offer flexibility to adapt to various financial time series forecasting requirements.

## 4. Case Studies  
Several case studies highlight the practical applications of these frameworks:
- **CryptoMamba**: This state space model demonstrates the successful application of long-range dependency modeling to Bitcoin price prediction compared to simpler methods, showcasing enhanced prediction accuracy under varying market conditions ([arXiv](https://arxiv.org/abs/2501.01010?utm_source=openai)).  
- **SynthBTC**: Leveraging Monte Carlo simulations, this tool provides diverse price scenario generation, which helps traders in decision-making processes based on potential price movements ([GitHub - synthBTC](https://github.imc.re/jofpin/synthBTC?utm_source=openai)).  

## 5. Challenges and Limitations  
Despite their advanced nature, these frameworks face ongoing challenges:
- **Market Volatility**: The unpredictable nature of Bitcoin markets can lead to reduced efficacy of models trained on historical data. Real-time adjustment of models is often required.
- **Data Quality**: The quality of input data (historical price and sentiment) directly influences model performance; hence, data cleansing and preprocessing are crucial steps.
- **Overfitting**: Complex models, especially deep learning frameworks, can become prone to overfitting, necessitating robust validation techniques like cross-validation.

## 6. Conclusion  
Open-source frameworks for Bitcoin price prediction employ a rich blend of statistical methods, machine learning, and deep learning paradigms. Each framework has unique strengths that cater to different analytical needs and practical applications. The constant evolution in this field emphasizes the necessity for ongoing research and adaptation to emerging market trends.

## 7. Future Directions  
Future research should focus on integrating frameworks to improve prediction accuracy through collaborative approaches. Advancements in machine learning, particularly in the fields of reinforcement learning and neural architecture search, may also contribute to improved predictions. Extending the scope of analysis to include other cryptocurrencies could provide broader insights into market dynamics.

## 8. References  
- Nixtla Documentation: [TimeGPT](https://nixtla.dev/docs/use-cases-bitcoin_price_prediction?utm_source=openai)  
- GitHub Repositories: [MRC-LSTM](https://arxiv.org/abs/2105.00707?utm_source=openai), [CryptoMamba](https://arxiv.org/abs/2501.01010?utm_source=openai)  
- Research Articles: [SARIMA and Prophet](https://www.sciencedirect.com/science/article/abs/pii/S0040162523006236?utm_source=openai)  
- Various frameworks as referenced above.

### As always, take a look at the trace

https://platform.openai.com/traces